# 電子書網頁標題翻頁爬蟲

In [ ]:
import requests
from bs4 import BeautifulSoup
for i in range(1,50): #第1頁到第49頁
    i=str(i)          #轉成字串
    url="http://www.allitebooks.org/page/"+i
    print(url)
    resp = requests.get(url)
    soup = BeautifulSoup(resp.text, 'lxml')
    titles = soup.find_all('h2')
    for t in titles:
        print(t.string)

# 爬取博客來圖書資訊

In [ ]:
#Ch9_2(books_crawler.py)-爬取博客來圖書資訊
import time
import requests
import csv
from bs4 import BeautifulSoup

# 目標URL網址
URL = "http://search.books.com.tw/search/query/key/{0}/cat/all"

def generate_search_url(url, keyword): #參數基底為URL和關鍵字
    url = url.format(keyword) #呼叫format函數建立搜尋圖書關鍵字的網址
    
    return url

def get_resource(url):
    headers = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
               "AppleWebKit/537.36 (KHTML, like Gecko)"
               "Chrome/63.0.3239.132 Safari/537.36"}
    return requests.get(url, headers=headers)


def parse_html(r):
    if r.status_code == requests.codes.ok:
        r.encoding = "utf8"
        soup = BeautifulSoup(r.text, "lxml")        
    else:
        print("HTTP請求錯誤..." + url)
        soup = None
    
    return soup    

#取得ISBN(國際標準書號)
def get_ISBN(url):
    soup = parse_html(get_resource("http:" + url))
    if soup != None:
        #try/elas:取得圖書的ISBN，如果找到就回傳ISBN碼，沒找到就回傳0000，剖析失敗則回傳1111
        try:
           isbn = soup.find(itemprop="productID")["content"][5:]
        except:
           isbn = "0000"
    else:
        isbn = "1111"
    return isbn

def save_to_csv(items, file):
    with open(file, 'w+', newline='') as fp:
        writer = csv.writer(fp)
        for item in items:
            writer.writerow(item)
            
def web_scraping_bot(url):
    booklist = [["書名","ISBN","網址","書價"]]
    print("抓取網路資料中...")
    soup = parse_html(get_resource(url))  #因為此爬蟲程式需要從多頁網頁取得資料，所以判斷HTTP請求是否成功移至parse_html函數
    if soup != None:
        # print(soup)
        tag_item = soup.find_all(class_="item") #取出所有圖書清單的HTML標籤
        for item in tag_item: #使用for迴圈一一爬取每一本圖書資料
            book = []
            book.append(item.find("img")["alt"])
            book.append(get_ISBN(item.find("a")["href"]))
            book.append("http:" + item.find("a")["href"])
            price = item.find(class_="price").find_all("b")
            if len(price) == 1: #判斷定價是否存在(因為可能有定價或優惠價)
                book.append(price[0].string)
            else:
                book.append(price[1].string)
            booklist.append(book) #在取得圖書資訊book清單後，呼叫append函數新增至booklist巢狀清單
            print("等待2秒鐘...")
            time.sleep(2) 

    return booklist

if __name__ == '__main__':
    url = generate_search_url(URL, "演算法")
    print(url)
    booklist = web_scraping_bot(url)
    for item in booklist:
        print(item)
    save_to_csv(booklist, "booklist.csv")
